In [ ]:
from typing import List
from collections import defaultdict
import json

game_node_ids: List[str] = []
user_node_ids: List[str] = []
edges_details_by_review_id: dict[str, dict] = {}
edges_review_id_user_id_pairs_by_game_id: dict[str, List[tuple[str, str]]] = defaultdict(list)
edges_review_id_game_id_pairs_by_user_id: dict[str, List[tuple[str, str]]] = defaultdict(list)

In [2]:
with open('../steam-applist-scraper/filtered_apps_ids.json', 'r') as f:
  filtered_app_ids = json.load(f)
  filtered_app_ids = [int(appid) for appid in filtered_app_ids]

game_nodes_ids = filtered_app_ids

In [3]:
import os

directory = '../steam-review-scraper/data'
game_ids_with_review_file = sorted([(filename.split("_")[1].split(".")[0]) for filename in os.listdir(directory)])

print(game_ids_with_review_file)

['10', '100', '1002', '100400', '10090', '10100', '10110', '10120', '10130', '10140', '10150', '10180', '10190', '10220', '102200', '10230', '10240', '102400', '10250', '102500', '10260', '102600', '10270', '102810', '102840', '102850', '104000', '104020', '104100', '10460', '104600', '1048580', '1048640', '1048660', '1048690', '1048710', '1048740', '1048780', '1048790', '1048850', '1048860', '1048890', '1048900', '1048920', '1048950', '1048960', '104900', '1049010', '1049030', '1049070', '1049080', '1049090', '1049100', '1049130', '1049140', '1049220', '1049270', '1049280', '1049310', '1049320', '1049340', '1049370', '1049380', '1049410', '1049510', '1049530', '1049560', '1049570', '1049630', '1049670', '1049680', '1049690', '1049710', '1049720', '1049730', '1049820', '1049840', '1049850', '1049870', '1049890', '1049910', '1049930', '1049950', '10500', '105000', '1050000', '1050010', '1050130', '1050150', '1050160', '1050170', '1050180', '1050210', '1050230', '1050240', '1050260', '10

In [ ]:
import gc
from tqdm import tqdm

for appid in tqdm(game_ids_with_review_file):
  # if (appid > 2000):
  #   break

  gc.collect()

  with open(f'../steam-review-scraper/data/review_{appid}.json', 'r') as f:
    reviews = json.load(f)
    for review_id, review in reviews['reviews'].items():
      user_id = review['author']['steamid']

      user_node_ids.append(user_id)
      game_node_ids.append(appid)
      edges_details_by_review_id[review_id] = {k: review[k] for k in ('voted_up', 'received_for_free')}

      edges_review_id_game_id_pairs_by_user_id[user_id].append((review_id, appid))
      edges_review_id_user_id_pairs_by_game_id[appid].append((review_id, user_id))

  0%|          | 0/17832 [00:00<?, ?it/s]

 39%|███▉      | 7028/17832 [9:52:17<30:30:51, 10.17s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7baad8e2f020>>
Traceback (most recent call last):
  File "/home/lockn/Projects/mc859-game-market-concentration-with-steam-data/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
 39%|███▉      | 7029/17832 [9:52:28<31:12:54, 10.40s/it]

In [ ]:
with open('game_node_ids_optimized.json', 'w') as f:
  json.dump(game_node_ids, f)

with open('user_node_ids_optimized.json', 'w') as f:
  json.dump(user_node_ids, f)
  
with open('edges_details_by_review_id_optimized.json', 'w') as f:
  json.dump(edges_details_by_review_id, f)

with open('edges_review_id_user_id_pairs_by_game_id_optimized.json', 'w') as f:
  json.dump(edges_review_id_user_id_pairs_by_game_id, f)

with open('edges_review_id_game_id_pairs_by_user_id_optimized.json', 'w') as f:
  json.dump(edges_review_id_game_id_pairs_by_user_id, f)

In [ ]:
import pandas as pd

pd.DataFrame({ 'game_node_id': game_node_ids }).to_csv('game_node_ids.csv', index=False)
pd.DataFrame({ 'user_node_id': user_node_ids }).to_csv('user_node_ids.csv', index=False)
pd.DataFrame([ { 'review_id': k, **v } for k, v in edges_details_by_review_id.items() ]).to_csv('edges_details_by_review_id.csv', index=False)
pd.DataFrame([ { 'game_id': k, 'review_id': review_id, 'user_id': user_id } for k, v in edges_review_id_user_id_pairs_by_game_id.items() for review_id, user_id in v ]).to_csv('edges_review_id_user_id_pairs_by_game_id.csv', index=False)
pd.DataFrame([ { 'user_id': k, 'review_id': review_id, 'game_id': game_id } for k, v in edges_review_id_game_id_pairs_by_user_id.items() for review_id, game_id in v ]).to_csv('edges_review_id_game_id_pairs_by_user_id.csv', index=False)
